In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:
print(train_data.index)
print(len(train_data))

In [ ]:
train_data.describe()

In [ ]:
test_data.describe()

In [ ]:
test_data[test_data["Parch"] > 5]

後ほど、"Parch" の値 9 は外れ値であると看做して、6に置き換える。<br>
Later, the value 9 for "Parch" shall be considered an outlier and replaced with 6.

In [ ]:
train_data.isnull()

In [ ]:
train_data.shape

In [ ]:
train_data.isnull().any()

In [ ]:
train_data.isnull().any(axis=1)

In [ ]:
train_data.isnull().any(axis=1).sum()

In [ ]:
train_data.loc[:, "Age"]

In [ ]:
print(train_data.loc[:, "Age"].isnull().sum())
print(train_data.loc[:, "Cabin"].isnull().sum())
print(train_data.loc[:, "Embarked"].isnull().sum())

In [ ]:
train_data[train_data.loc[:, "Embarked"].isnull()]

In [ ]:
train_data[train_data.loc[:, "Age"].isnull()]

In [ ]:
train_data["Age"].sort_values()

In [ ]:
processed_train_data = train_data.loc[:, ["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
print(processed_train_data.dtypes)
processed_train_data = processed_train_data.replace({"male": 0, "female": 1})
processed_train_data = processed_train_data.replace({"C": 0, "Q": 1, "S": 2})
processed_train_data["Pclass"] -= 1
processed_train_data["Age"] = processed_train_data["Age"].mask(processed_train_data["Age"] < 1, 0)
print(processed_train_data.dtypes)
processed_train_data

In [ ]:
processed_test_data = test_data.loc[:, ["PassengerId", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
print(processed_test_data.dtypes)
processed_test_data = processed_test_data.replace({"male": 0, "female": 1})
processed_test_data = processed_test_data.replace({"C": 0, "Q": 1, "S": 2})
processed_test_data ["Pclass"] -= 1
processed_test_data["Age"] = processed_test_data["Age"].mask(processed_test_data["Age"] < 1, 0)
print(processed_test_data.dtypes)
processed_test_data

In [ ]:
processed_train_data.describe()

In [ ]:
processed_test_data.describe()

In [ ]:
processed_train_data.isnull().any()

In [ ]:
print(processed_test_data.isnull().any())
processed_test_data[processed_test_data["Fare"].isnull()]

In [ ]:
processed_test_data["Parch"][processed_test_data["Parch"] == 9]

In [ ]:
# The value 9 for "Parch" shall be considered an outlier and replaced with 6.
processed_test_data.loc[342, "Parch"] = 6
processed_test_data.loc[365, "Parch"] = 6
processed_test_data.describe()

後で、SibSp, Fare, Parch は正規化する。<br>
Later, normalize the features SibSp, Fare and Parch.

In [ ]:
train_data[train_data["Fare"] == 0.0]

In [ ]:
test_data[test_data["Fare"] == 0.0]

運賃 0？　この不思議な乗客たちはどうすべきか。<br>
Fare 0?  What to do with this strange passengers.

In [ ]:
train_data["Fare"].hist(by=train_data["Pclass"], bins=100)

In [ ]:
test_data["Fare"].hist(by=test_data["Pclass"], bins=100)

In [ ]:
print(processed_train_data.groupby("Pclass")["Fare"].apply(lambda x: x.mode()))
#print(processed_train_data.groupby("Pclass")["Fare"].mode()) # Why don't work?
processed_train_data.groupby("Pclass")["Fare"].describe()

In [ ]:
print(processed_test_data.groupby("Pclass")["Fare"].apply(lambda x: x.mode()))
processed_test_data.groupby("Pclass")["Fare"].describe()

In [ ]:
len(train_data[train_data["Fare"] == 0.0])

運賃 0 は集計のミスではないかと疑い、補正することを考えたが、15人中14人が死亡しているのは顕著な特徴と言える。運賃 0 は集計のミスによるものではなく、何か特殊な乗客を示す属性と考えられるので、このまま採用する。<br>

I suspected that the fare 0 might be an error in tabulation and considered correcting for it, but 14 of the 15 deaths is a remarkable features.  Since fare 0 is not due to an error in tabulation, but is considered an attribute that indicates some special passenger, it is adopted as is.

In [ ]:
processed_train_data.corr(method="pearson")

In [ ]:
import matplotlib.pyplot as plt
from pylab import rcParams

In [ ]:
rcParams["figure.figsize"] = 6, 4
heatmap = plt.imshow(processed_train_data.corr(method="pearson"), cmap="bwr", vmax=1, vmin=-1)
plt.colorbar(heatmap)
# Survived Pclass Sex Age SibSp Parch Fare Embarked

In [ ]:
processed_train_data_dropna = processed_train_data.dropna(subset=["Age"])
print(len(processed_train_data_dropna))
processed_train_data_dropna.isnull().any()

"Age" には NA値が多いため、"Age" に NA値を持つ行を削除して相関に変化があるかを検証。結果、ほとんど変化なし。<br>
"Age" has many NA values, so we removed rows with NA values in "Age" and verified if the correlation changed. The result was almost no change.

In [ ]:
processed_train_data_dropna.corr(method="pearson")

In [ ]:
heatmap2 = plt.imshow(processed_train_data_dropna.corr(method="pearson"), cmap="bwr", vmax=1, vmin=-1)
plt.colorbar(heatmap2)

Replace the age of children under 1 year old with 0.

In [ ]:
rcParams["figure.figsize"] = 10, 10
processed_train_data.hist()

In [ ]:
rcParams["figure.figsize"] = 6, 4
processed_train_data["Age"].hist(bins=81)

In [ ]:
processed_train_data["Age"].hist(by=processed_train_data["Survived"], bins=81)

In [ ]:
processed_train_data["Survived"].hist(by=processed_train_data["Sex"])

In [ ]:
rcParams["figure.figsize"] = 7, 5
processed_train_data["Survived"].hist(by=processed_train_data["Pclass"])

In [ ]:
rcParams["figure.figsize"] = 8, 8
processed_train_data["Survived"].hist(by=processed_train_data["Embarked"])

In [ ]:
rcParams["figure.figsize"] = 8, 8
processed_train_data["Survived"].hist(by=processed_train_data["Embarked"])

In [ ]:
processed_train_data["Pclass"].hist(by=processed_train_data["Embarked"])

シェルブールから乗船する乗客だけ生存率が高いのは、おそらくシェルブールからの乗客の多くが上流階級だからだろう。<br>
The survival rate is high only for passengers embarking from Cherbourg, perhaps because many of the passengers from Cherbourg are upper class.

In [ ]:
rcParams["figure.figsize"] = 10, 10
processed_train_data["Survived"].hist(by=processed_train_data["SibSp"])

In [ ]:
processed_train_data["Survived"].hist(by=processed_train_data["Parch"])

In [ ]:
rcParams["figure.figsize"] = 20, 20
processed_train_data.hist("Fare", bins=50)

In [ ]:
print(processed_train_data[processed_train_data["Fare"] > 300])

彼らは特別な富裕層。<br>
They are a special wealthy group.

In [ ]:
print(processed_train_data[processed_train_data["Fare"] > 200].sort_values("Fare"))

金持ちは助かる率が高そうである。<br>
The rich seem to have a higher rate of survival.

In [ ]:
import torch

In [ ]:
processed_train_data.describe()

SibSp, Fare, Parch を正規化する。<br>
Normalize SibSp, Fare, and Parch.

In [ ]:
processed_train_data["SibSp"] = (processed_train_data["SibSp"] - processed_train_data["SibSp"].min()) / (processed_train_data["SibSp"].max() - processed_train_data["SibSp"].min())
processed_train_data["Fare"] = (processed_train_data["Fare"] - processed_train_data["Fare"].min()) / (processed_train_data["Fare"].max() - processed_train_data["Fare"].min())
processed_train_data["Parch"] = (processed_train_data["Parch"] - processed_train_data["Parch"].min()) / (processed_train_data["Parch"].max() - processed_train_data["Parch"].min())
print("done")

In [ ]:
processed_test_data["SibSp"] = (processed_test_data["SibSp"] - processed_test_data["SibSp"].min()) / (processed_test_data["SibSp"].max() - processed_test_data["SibSp"].min())
processed_test_data["Fare"] = (processed_test_data["Fare"] - processed_test_data["Fare"].min()) / (processed_test_data["Fare"].max() - processed_test_data["Fare"].min())
processed_test_data["Parch"] = (processed_test_data["Parch"] - processed_test_data["Parch"].min()) / (processed_test_data["Parch"].max() - processed_test_data["Parch"].min())
print("done")

In [ ]:
processed_train_data.describe()

In [ ]:
processed_test_data.describe()

In [ ]:
processed_train_data.hist()

In [ ]:
processed_test_data.hist()

In [ ]:
def one_hot_encode(data, size):
    n = int(size)
    I = np.eye(n)
    ret_list = []
    
    for datum in data:
        if np.isnan(datum):
            ret_list.append(np.zeros(n))
        else:
            ret_list.append(I[int(datum)])
    
    ret_list = np.array(ret_list)
    return torch.tensor(ret_list, dtype=torch.float)

In [ ]:
pcls = one_hot_encode(processed_train_data["Pclass"], processed_train_data["Pclass"].max() + 1)
age = one_hot_encode(processed_train_data["Age"], processed_train_data["Age"].max() + 1)
emb = one_hot_encode(processed_train_data["Embarked"], processed_train_data["Embarked"].max() + 1)
print(pcls.shape, age.shape, emb.shape)

In [ ]:
# The size should match that of train_data.
test_pcls = one_hot_encode(processed_test_data["Pclass"], processed_train_data["Pclass"].max() + 1)
test_age = one_hot_encode(processed_test_data["Age"], processed_train_data["Age"].max() + 1)
test_emb = one_hot_encode(processed_test_data["Embarked"], processed_train_data["Embarked"].max() + 1)
print(test_pcls.shape, test_age.shape, test_emb.shape)

In [ ]:
label = torch.tensor(processed_train_data["Survived"], dtype=torch.float).unsqueeze(1)
sex = torch.tensor(processed_train_data["Sex"], dtype=torch.float).unsqueeze(1)
sibsp = torch.tensor(processed_train_data["SibSp"], dtype=torch.float).unsqueeze(1)
parch = torch.tensor(processed_train_data["Parch"], dtype=torch.float).unsqueeze(1)
fare = torch.tensor(processed_train_data["Fare"], dtype=torch.float).unsqueeze(1)
print(label.shape, sex.shape, sibsp.shape, parch.shape, fare.shape)

In [ ]:
test_pid = torch.tensor(processed_test_data["PassengerId"], dtype=torch.float).unsqueeze(1)
test_sex = torch.tensor(processed_test_data["Sex"], dtype=torch.float).unsqueeze(1)
test_sibsp = torch.tensor(processed_test_data["SibSp"], dtype=torch.float).unsqueeze(1)
test_parch = torch.tensor(processed_test_data["Parch"], dtype=torch.float).unsqueeze(1)
test_fare = torch.tensor(processed_test_data["Fare"], dtype=torch.float).unsqueeze(1)
print(test_pid.shape, test_sex.shape, test_sibsp.shape, test_parch.shape, test_fare.shape)

In [ ]:
train_X =  torch.cat([label, pcls, sex, age, sibsp, parch, fare, emb], dim=1)
train_X.shape

In [ ]:
test_X = torch.cat([test_pid, test_pcls, test_sex, test_age, test_sibsp, test_parch, test_fare, test_emb], dim=1)
test_X.shape

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_size = int(train_X.shape[0] * 0.85)
val_size = train_X.shape[0] - train_size
print(train_size, val_size)

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
class TitanicNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Linear(91, 91)
        self.norm1 = nn.BatchNorm1d(91)
        self.drop1 = nn.Dropout(0.9)
        
        self.layer2 = nn.Linear(91, 91)
        self.norm2 = nn.BatchNorm1d(91)
        self.drop2 = nn.Dropout(0.9)
        
        self.layer3 = nn.Linear(91, 40)
        self.norm3 = nn.BatchNorm1d(40)
        self.drop3 = nn.Dropout(0.9)
        
        self.layer4 = nn.Linear(40, 1)
        

    
    def forward(self, x):
        skip = x

        x = self.layer1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.drop1(x)
        
        x = self.layer2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.drop2(x)
        
        x = self.layer3(x + skip)
        x = self.norm3(x)
        x = self.drop3(x)
        
        x = self.layer4(x)
        
        return x

In [ ]:
train_set, val_set = torch.utils.data.random_split(train_X, [train_size, val_size])

batch_size = 32
train_loader = DataLoader(train_set, batch_size=757, shuffle=True)
val_loader = DataLoader(val_set, batch_size=134)

model = TitanicNet()

learning_rate = 0.0005
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

from torchinfo import summary
summary(model)

In [ ]:
from tqdm.notebook import tqdm
epochs = 3000

train_loss_hist = []
train_acc_hist = []
val_loss_hist = []
val_acc_hist = []
pred_hist = []

for epoch in range(epochs):
        
    for batch in train_loader:
        model.train()
        labels = batch[:, 0].unsqueeze(1)
        #labels = torch.ones_like(labels)
        X = batch[:, 1:]

        optimizer.zero_grad()
        pred = model(X)
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            temp_loss = loss.item()
            train_loss_hist.append(temp_loss)
            pred_hist.append(torch.round(F.sigmoid(pred)))
            acc = (torch.round(F.sigmoid(pred)) == labels).sum() / len(labels)
            acc=acc.item()
            train_acc_hist.append(acc)

    with torch.no_grad():
        
        for val_batch in val_loader:
            model.eval()
            labels = val_batch[:, 0].unsqueeze(1)
            X = val_batch[:, 1:]
            pred = model(X)
            val_loss = criterion(pred, labels)

            val_temp_loss = val_loss.item()
            val_loss_hist.append(val_temp_loss)
            val_acc = (torch.round(F.sigmoid(pred)) == labels).sum() / len(labels)
            val_acc = val_acc.item()
            val_acc_hist.append(val_acc)

    if epoch % 100 == 0:
        print(f"epoch:{epoch}, train_loss={temp_loss}, validation_loss={val_temp_loss}")

In [ ]:
rcParams["figure.figsize"] = 6, 4
plt.plot(train_loss_hist)

In [ ]:
plt.plot(train_acc_hist)

In [ ]:
plt.plot(val_loss_hist)

In [ ]:
plt.plot(val_acc_hist)

700エポックくらいで止めてみるか。<br>
I'd stop at about 700 epochs.

In [ ]:
train_set, val_set = torch.utils.data.random_split(train_X, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=757, shuffle=True)
val_loader = DataLoader(val_set, batch_size=134)

model = TitanicNet()

learning_rate = 0.0005
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

from torchinfo import summary
summary(model)

In [ ]:
from tqdm.notebook import tqdm
epochs = 650

train_loss_hist = []
train_acc_hist = []
val_loss_hist = []
val_acc_hist = []
pred_hist = []

for epoch in range(epochs):
        
    for batch in train_loader:
        model.train()
        labels = batch[:, 0].unsqueeze(1)
        #labels = torch.ones_like(labels)
        X = batch[:, 1:]

        optimizer.zero_grad()
        pred = model(X)
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            temp_loss = loss.item()
            train_loss_hist.append(temp_loss)
            pred_hist.append(torch.round(F.sigmoid(pred)))
            acc = (torch.round(F.sigmoid(pred)) == labels).sum() / len(labels)
            acc=acc.item()
            train_acc_hist.append(acc)

    with torch.no_grad():
        
        for val_batch in val_loader:
            model.eval()
            labels = val_batch[:, 0].unsqueeze(1)
            X = val_batch[:, 1:]
            pred = model(X)
            val_loss = criterion(pred, labels)

            val_temp_loss = val_loss.item()
            val_loss_hist.append(val_temp_loss)
            val_acc = (torch.round(F.sigmoid(pred)) == labels).sum() / len(labels)
            val_acc = val_acc.item()
            val_acc_hist.append(val_acc)

    if epoch % 100 == 0:
        print(f"epoch:{epoch}, train_loss={temp_loss}, validation_loss={val_temp_loss}")

In [ ]:
plt.plot(val_loss_hist)

In [ ]:
plt.plot(val_acc_hist)

悪くない。<br>
Not bad.

In [ ]:
print(test_X.shape)
test_loader = DataLoader(test_X, batch_size=418)

In [ ]:
with torch.no_grad():

    for test_batch in test_loader:
        model.eval()
        pid = test_batch[:, 0].unsqueeze(1)
        X = test_batch[:, 1:]
        pred = model(X)
        pred = torch.round(F.sigmoid(pred))
    
print("done")

In [ ]:
print(pid.shape, pred.shape)

In [ ]:
pid = pid.to(torch.int).numpy()
pred = pred.to(torch.int).numpy()

In [ ]:
data = {"PassengerId": pid.squeeze(), "Survived": pred.squeeze()}
submit = pd.DataFrame(data)
submit

In [ ]:
submit.to_csv('submission.csv', index=False)